In [1]:
from arrest.handler import ResourceHandler
from arrest import Service, Resource
import re

In [10]:
svc = Service(
    name="xyz",
    url="http://example.com",
    resources=[
        Resource(
            route="/user",
            handlers=[
                ("GET", "/posts/{foo:str}/comments/{baz:int}/"),
            ],
        )
    ],
)

In [11]:
svc.user.get_matching_handler(method="GET", path="/posts")